# Gesture Recognition

In [1]:
# Importing relevant libraries

import numpy as np
import os
from imageio.v2 import imread
import datetime
import os
import cv2

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

C:\Users\Leander Antony A\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In this block, we read the folder names for training and validation, also set the `batch_size` here.

In [3]:
# Reading the .csv files

train_doc = np.random.permutation(open("E:\\Documents\\AIML_masters\\course4-deeplearning\\week7\\Project_data\\train.csv").readlines())
val_doc = np.random.permutation(open("E:\\Documents\\AIML_masters\\course4-deeplearning\\week7\\Project_data\\val.csv").readlines())

batch_size = 32
image_ids = list(range(1, 30, 2))
num_classes = 5
num_images_used = len(image_ids)
image_width = 160
image_height = 160
channels = 3
image_ids, num_images_used

([1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29], 15)

In [4]:
# train and val dataset shape
train_doc.shape, val_doc.shape

((663,), (100,))

## Generator

we are going to preprocess the images and perform normalization to improve accuracy.Also a generator can now feed the data in batches instead of all at once to save memory.

In [5]:
# Defining the generator

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = image_ids
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size    # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,num_images_used,image_width,image_height,channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,num_classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    
                    temp = cv2.resize(image,(image_width,image_height))
                    temp = temp/255 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])  #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])  #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])  #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,num_images_used,image_width,image_height,channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,num_classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = cv2.resize(image,(image_width,image_height))
                    temp = temp/255 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

A video is represented above in the generator as (number of images, height, width, number of channels). 

In [6]:
# Assigning the path for train and val data, initialising the epoch number

curr_dt_time = datetime.datetime.now()
train_path = "E:\\Documents\\AIML_masters\\course4-deeplearning\\week7\\Project_data\\train"
val_path = "E:\\Documents\\AIML_masters\\course4-deeplearning\\week7\\Project_data\\val"
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20 # choosing the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model

building a 3D convolution model using different functionalities that Keras provides. Using `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. 

In [7]:
# Importing the relevant keras libraries

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, Input
from keras.layers import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

Building the model

In [8]:
input_shape = (15, 160, 160, 3)

# Initialize Sequential model
model = Sequential()

# Add input layer
model.add(Input(shape=input_shape))

# Add convolutional layers
model.add(Conv3D(32, (2, 2, 2), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(64, (3, 3, 3), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(128, (3, 3, 3), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Flatten the output
model.add(Flatten())

# Add fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))

# Output layer
num_classes = 5  
model.add(Dense(num_classes, activation='softmax'))

In [9]:
# Using the 'adam' optimiser and checking the summary of model

optimiser = 'adam' 
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                      │ (None, 15, 160, 160, 32)    │             800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d (MaxPooling3D)         │ (None, 7, 80, 80, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_1 (Conv3D)                    │ (None, 7, 80, 80, 64)       │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_1 (MaxPooling3D)       │ (None, 3, 40, 40, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_2 (Conv3D)                    │ (None, 3, 40, 40, 128)      │         221,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_2 (MaxPooling3D)       │ (None, 1, 20, 20, 128)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 51200)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       6,553,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,831,845 (26.06 MB)

 Trainable params: 6,831,845 (26.06 MB)

 Non-trainable params: 0 (0.00 B)

None


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [10]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [11]:
# Define the model name
curr_dt_time = datetime.datetime.now()
gesture_recog = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

# Create directory if it doesn't exist
if not os.path.exists(gesture_recog):
    os.mkdir(gesture_recog)
        
# Define the file path to save the model
filepath = gesture_recog + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras'

# Model checkpoint callback
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')

# Learning rate scheduler callback
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, mode='auto')

# List of callbacks
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it needs to make.

In [12]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, we'll be able to save the model at the end of each epoch.

In [13]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  E:\Documents\AIML_masters\course4-deeplearning\week7\Project_data\train ; batch size = 32
Epoch 1/20
19/21 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - categorical_accuracy: 0.2357 - loss: 2.3361Batch:  21 Index: 32
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - categorical_accuracy: 0.2342 - loss: 2.2927Source path =  E:\Documents\AIML_masters\course4-deeplearning\week7\Project_data\val ; batch size = 32
Batch:  4 Index: 32

Epoch 1: val_loss improved from inf to 1.50722, saving model to model_init_2024-04-1003_30_32.977834/model-00001-1.87421-0.22172-1.50722-0.24000.keras
21/21 ━━━━━━━━━━━━━━━━━━━━ 71s 3s/step - categorical_accuracy: 0.2336 - loss: 2.2737 - val_categorical_accuracy: 0.2400 - val_loss: 1.5072 - learning_rate: 0.0010
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - categorical_accuracy: 0.2950 - loss: 1.4561
Epoch 2: val_loss improved from 1.50722 to 1.25065, saving model to model_init_2024-04-1003_30_32.977834/model-00002-1.44533-0.30849-1.25065-0.50000.keras
21/21 ━━━

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - categorical_accuracy: 0.9788 - loss: 0.0629
Epoch 18: val_loss did not improve from 0.06340

Epoch 18: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
21/21 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - categorical_accuracy: 0.9783 - loss: 0.0637 - val_categorical_accuracy: 0.8125 - val_loss: 0.9475 - learning_rate: 0.0010
Epoch 19/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - categorical_accuracy: 0.9835 - loss: 0.0499
Epoch 19: val_loss did not improve from 0.06340
21/21 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - categorical_accuracy: 0.9836 - loss: 0.0496 - val_categorical_accuracy: 0.7500 - val_loss: 0.7121 - learning_rate: 1.0000e-04
Epoch 20/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - categorical_accuracy: 0.9853 - loss: 0.0473
Epoch 20: val_loss did not improve from 0.06340
21/21 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - categorical_accuracy: 0.9855 - loss: 0.0471 - val_categorical_accuracy: 0.9375 - val_loss: 0.4547 - learning_rate: 1.0000e-04
